This notebook is a supplement to the paper ["The Orthologic of Epistemic Modals"](https://escholarship.org/uc/item/0ss5z8g3) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu) and [Matthew Mandelkern](mandelkern@nyu.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations. For example, we treat the logical principle $\Box\varphi \vdash \varphi$, corresponding to the lattice inequality $\Box a\leq a$, as the equation $\Box a = \Box a\wedge a$.

In [1]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

### Lattice axioms

In [2]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

### Bounded lattice axioms

In [3]:
bot = read_expr('And(x,Bot) = Bot')
top = read_expr('Or(x,Top) = Top')

bounded_lattice = lattice + [bot,top]

### Ortholattice axioms

In [4]:
contra = read_expr('And(x,Not(x)) = Bot')
lem = read_expr('Or(x,Not(x)) = Top')

invol = read_expr('Not(Not(x)) = x')

de_morgan = read_expr('Not(And(x,y)) = Or(Not(x),Not(y))')

ortho_lattice = bounded_lattice + [contra,lem,invol,de_morgan]

In [6]:
#We do not want distributivity, but we consider it below
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

In [7]:
#Check that the De Morgan law dual to de_morgan above follows
goal = read_expr('Not(Or(x,y)) = And(Not(x),Not(y))')
prover = Prover9Command(goal, assumptions=ortho_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36399 was started by halcrow on iMac.local,
Fri Feb  4 11:37:00 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.02 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 70.


1 Not(Or(x,y)) = And(Not(x),Not(y)).  [goal].
16 Not(Not(x)) = x.  [assumption].
17 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
18 Not(Or(c1,c2)) != And(Not(c1),Not(c2)).  [deny(1)].
49 Not(Or(Not(x),Not(y))) = And(x,y).  [para(17(a,1),16(a,1,1))].
168 Not(Or(x,Not(y))) = And(Not(x),y).  [para(16(a,1),49(a,1,1,1))].
277 Not(Or(x,y)) = And(Not(

In [9]:
#Check that the distributive law does not follow from ortholattice axioms
goal = dist
mb = MaceCommand(goal, assumptions=ortho_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

c3 = 4.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 0.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(2,4) = 1.
Or(2,5) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.
Or(3,4) = 1.
Or(3,5) = 1.
Or(4,0)

### Boolean subalgebra axioms

In [10]:
B_bot = read_expr('B(Bot)')
B_not = read_expr('B(x) -> B(Not(x))')
B_and = read_expr('(B(x) & B(y)) -> B(And(x,y))')
B_or = read_expr('(B(x) & B(y)) -> B(Or(x,y))')
B_dist = read_expr('((B(x) & B(y)) & B(z)) -> And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

ortho_boolean_lattice = ortho_lattice + [B_bot, B_not, B_and, B_or, B_dist]

In [11]:
#Check that Top belongs to the Boolean subalgebra
goal = read_expr('B(Top)')
prover = Prover9Command(goal, assumptions=ortho_boolean_lattice)
prover.prove()
print(prover.proof())

#Check that the other distributive law dual to B_dist above follows
goal = read_expr('((B(x) & B(y)) & B(z)) -> Or(x,And(y,z)) = And(Or(x,y),Or(x,z))')
prover = Prover9Command(goal, assumptions=ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36405 was started by halcrow on iMac.local,
Fri Feb  4 11:37:23 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 13.
% Level of proof is 3.
% Maximum clause weight is 10.000.
% Given clauses 16.


1 B(x) -> B(Not(x)).  [assumption].
5 B(Top).  [goal].
8 Or(x,y) = Or(y,x).  [assumption].
18 And(x,Not(x)) = Bot.  [assumption].
19 Or(x,Not(x)) = Top.  [assumption].
20 Not(Not(x)) = x.  [assumption].
21 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
22 B(Bot).  [assumption].
23 -B(x) | B(Not(x)).  [clausify(1)].
28 -B(Top).  [deny(5)].
57 Not

### Modal axioms

In [12]:
box_and = read_expr('Box(And(x,y)) = And(Box(x),Box(y))')
box_top = read_expr('Box(Top) = Top')
diamond_dual = read_expr("Diamond(x) = Not(Box(Not(x)))")

modal_ortho_boolean_lattice = ortho_boolean_lattice + [box_and, box_top, diamond_dual]

In [13]:
#Check that Diamond distributes over disjunction
goal = read_expr('Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y))')
prover = Prover9Command(goal, assumptions=modal_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36411 was started by halcrow on iMac.local,
Fri Feb  4 11:37:39 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.04 (+ 0.01) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 16.000.
% Given clauses 107.


5 Diamond(Or(x,y)) = Or(Diamond(x),Diamond(y)).  [goal].
20 Not(Not(x)) = x.  [assumption].
21 Not(And(x,y)) = Or(Not(x),Not(y)).  [assumption].
28 Box(And(x,y)) = And(Box(x),Box(y)).  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
31 Diamond(Or(c1,c2)) != Or(Diamond(c1),Diamond(c2)).  [deny(5)].
32 Not(Box(Not(Or(c1,c2)))) != Or(N

### Epistemic axioms

In [14]:
factive = read_expr('Box(x) = And(Box(x),x)')
episteme = read_expr('And(x,Diamond(Not(x))) = Bot')

epistemic_ortho_boolean_lattice = modal_ortho_boolean_lattice + [factive, episteme]

In [15]:
#Check that another form of Wittgenstein sentence is contradictory
goal = read_expr('And(Not(x),Diamond(x)) = Bot')
prover = Prover9Command(goal, assumptions=epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

#Check that "p or it must be that not p" is derivable
goal = read_expr('Or(x,Box(Not(x))) = Top')
prover = Prover9Command(goal, assumptions=epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36415 was started by halcrow on iMac.local,
Fri Feb  4 11:37:47 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 2.
% Maximum clause weight is 7.000.
% Given clauses 0.


5 And(Not(x),Diamond(x)) = Bot.  [goal].
20 Not(Not(x)) = x.  [assumption].
30 Diamond(x) = Not(Box(Not(x))).  [assumption].
33 And(x,Diamond(Not(x))) = Bot.  [assumption].
34 And(x,Not(Box(x))) = Bot.  [copy(33),rewrite([30(2),20(2)])].
35 And(Not(c1),Diamond(c1)) != Bot.  [deny(5)].
36 $F.  [copy(35),rewrite([30(4),34(7)]),xx(a)].

=================

In [16]:
#Check that Diamond p does not entail p
goal = read_expr('Diamond(x) = And(Diamond(x),x)')
mb = MaceCommand(goal, assumptions=epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 5.
Box(4) = 2.
Box(5) = 5.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 4.
Diamond(3) = 3.
Diamond(4) = 4.
Diamond(5) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 2.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 5.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 2.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 5.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2

### Conditional axioms

In [17]:
if_and = read_expr('If(x,And(y,z)) = And(If(x,y),If(x,z))')
if_top = read_expr('If(x,Top) = Top')

ident = read_expr('If(x,x) = Top') 

simp_mp = read_expr('B(y) -> And(If(x,y),x) = And(And(If(x,y),x),y)')
simp_cs = read_expr('B(y) -> And(x,y) = And(And(x,y),If(x,y))')
simp_mt = read_expr('B(y) -> And(If(x,y),Not(y)) = And(And(If(x,y),Not(y)),Not(x))')

stm = read_expr('If(x,y) = And(If(x,y),Or(Not(x),y))')

simp_caut_trans = read_expr('B(z) -> And(If(x,y),If(And(x,y),z)) = And(And(If(x,y),If(And(x,y),z)),If(x,z))')
caut_mon = read_expr('And(If(x,y),If(x,z)) = And(And(If(x,y),If(x,z)),If(And(x,y),z))')

flat = read_expr('If(x,If(And(x,y),z)) = If(And(x,y),z)')

simp_cem_plus = read_expr('B(x) -> If(x,Or(y,z)) = And(If(x,Or(y,z)),Or(If(x,y),If(x,z)))')
simp_ni = read_expr('B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y)))')

weak_boethius = read_expr('And(Diamond(x),If(x,y)) = And(And(Diamond(x),If(x,y)),Not(If(x,Not(y))))')

must_intro = read_expr('x = And(x,y) -> If(x,Box(y))=1')
must_preserve = read_expr('B(y) -> And(Diamond(x),Box(y)) = And(And(Diamond(x),Box(y)),If(x,Box(y)))')


cond_ax = [if_and, if_top, ident, 
           simp_mp, simp_cs, simp_mt, 
           stm, simp_caut_trans, caut_mon, 
           flat, simp_cem_plus, simp_ni,
           weak_boethius, must_intro, must_preserve]

cond_epistemic_ortho_boolean_lattice = epistemic_ortho_boolean_lattice + cond_ax

Next we show that if $\varphi\vdash\psi$, then $\psi\to\bot\vdash \varphi\to\bot$ (Absurdity Reversal).

In [18]:
goal = read_expr('x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot)) ')
prover = Prover9Command(goal, assumptions=cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36426 was started by halcrow on iMac.local,
Fri Feb  4 11:38:09 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.48 (+ 0.02) seconds.
% Length of proof is 64.
% Level of proof is 10.
% Maximum clause weight is 24.000.
% Given clauses 541.


1 B(x) -> B(Not(x)).  [assumption].
5 B(x) -> And(If(y,x),y) = And(And(If(y,x),y),x).  [assumption].
10 B(x) -> Not(If(x,y)) = And(Not(If(x,y)),If(x,Not(y))).  [assumption].
13 x = And(x,y) -> If(y,Bot) = And(If(y,Bot),If(x,Bot)).  [goal].
16 Or(x,y) = Or(y,x).  [assumption].
17 And(x,y) = And(y,x).  [assumption].
18 Or(x,Or(y,z)) = Or(Or(x,y),z)

Next we show $(\varphi\to\bot)\vdash \neg\varphi$ (Ad Falsum).

In [19]:
goal = read_expr('If(x,Bot) = And(If(x,Bot),Not(x))')
prover = Prover9Command(goal, assumptions=cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 36428 was started by halcrow on iMac.local,
Fri Feb  4 11:38:14 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 22.
% Level of proof is 4.
% Maximum clause weight is 18.000.
% Given clauses 30.


7 B(x) -> And(If(y,x),Not(x)) = And(And(If(y,x),Not(x)),Not(y)).  [assumption].
13 If(x,Bot) = And(If(x,Bot),Not(x)).  [goal].
16 Or(x,y) = Or(y,x).  [assumption].
17 And(x,y) = And(y,x).  [assumption].
23 And(x,Or(x,y)) = x.  [assumption].
25 Or(x,Top) = Top.  [assumption].
26 And(x,Not(x)) = Bot.  [assumption].
27 Or(x,Not(x)) = Top.  [assumption

Next we show that $\alpha\to\beta \not\equiv \neg\alpha\vee\beta $ (No Simple Collapse).

In [20]:
goal = read_expr('B(x) & B(y) -> If(x,y) = Or(Not(x),y)')
mb = MaceCommand(goal, assumptions= cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

c2 = 0.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

  B(0).
  B(1).
  B(2).
  B(3).



Next we show that $\Diamond\alpha\wedge\beta\nvdash \alpha\to\beta$.

In [21]:
goal = read_expr('B(x) & B(y) -> And(M(x),y) = And(And(M(x),y),If(x,y))')
mb = MaceCommand(goal, assumptions= cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.

M(0) = 0.
M(1) = 0.
M(2) = 1.
M(3) = 0.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

  B(0).
  B(1).
  B(2).
  B(3).



Next we show that $\nvdash \neg \beta\vee (\alpha\to\beta)$.

In [22]:
goal = read_expr('B(x) & B(y) -> Or(Not(y),If(x,y)) = Top')
mb = MaceCommand(goal, assumptions= cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 4

Bot = 0.

Top = 1.

c1 = 2.

c2 = 3.

Box(0) = 0.
Box(1) = 1.
Box(2) = 2.
Box(3) = 3.

Diamond(0) = 0.
Diamond(1) = 1.
Diamond(2) = 2.
Diamond(3) = 3.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.

If(0,0) = 1.
If(0,1) = 1.
If(0,2) = 1.
If(0,3) = 1.
If(1,0) = 0.
If(1,1) = 1.
If(1,2) = 2.
If(1,3) = 3.
If(2,0) = 0.
If(2,1) = 1.
If(2,2) = 1.
If(2,3) = 0.
If(3,0) = 0.
If(3,1) = 1.
If(3,2) = 0.
If(3,3) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.

  B(0).
  B(1).
  B(2).
  B(3).



### Principles for which Prover9/Mace4 does not produce a proof/counterexample

Consider $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$ (Conditional Modus Tollens), which is provable as follows:

1. $\psi\to\bot \vdash \psi\to\varphi$ and $\psi\to\bot\vdash\psi\to\bot$, so $\psi\to\bot \vdash (\psi\to\varphi)\wedge (\psi\to\bot)$
2. By Cautious Monotonicity, $(\psi\to\varphi)\wedge (\psi\to\bot)\vdash (\psi\wedge\varphi)\to\bot\vdash (\varphi\wedge\psi)\to\bot$.
3. By 1 and 2, $\psi\to\bot \vdash (\varphi\wedge\psi)\to\bot$.
4. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge (\varphi\wedge\psi)\to\bot\vdash \varphi\to\bot$.
5. By 3 and 4, $(\varphi\to\psi)\wedge (\psi\to\bot)\vdash \varphi\to\bot$.

Unfortunately Prover9/Mace4 does not find a proof/counterexample.

In [ ]:
goal = read_expr('And(If(x,y),If(y,Bot)) = And(And(If(x,y),If(y,Bot)),If(x,Bot))')
prover = Prover9Command(goal, assumptions=ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,caut_mon])
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr('And(If(x,y),If(y,Bot)) = And(And(If(x,y),If(y,Bot)),If(x,Bot))')
mb = MaceCommand(goal, assumptions=ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,caut_mon])
mb.build_model()
print(mb.model(format='cooked'))

Consider, for non-epistemic $\beta$, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$ (Simple Reciprocity), which is provable as follows:

1. By Cautious Monotonicity,  $(\psi\to\varphi)\wedge (\psi\to\beta)\vdash (\psi\wedge\varphi)\to\beta\vdash (\varphi\wedge\psi)\to\beta$.

2. By Simple Cautious Transitivity, $(\varphi\to\psi)\wedge ((\varphi\wedge\psi)\to\beta)\vdash \varphi\to\beta$.

3. By 1 and 2, $(\varphi\to\psi)\wedge (\psi\to\varphi)\wedge (\psi\to\beta)\vdash \varphi\to\beta$.


In [ ]:
goal = read_expr('B(z) -> And(And(If(x,y),If(y,x)),If(y,z)) = And(And(And(If(x,y),If(y,x)),If(y,z)),If(x,z))')
prover = Prover9Command(goal, assumptions=ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,caut_mon])
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr('B(z) -> And(And(If(x,y),If(y,x)),If(y,z)) = And(And(And(If(x,y),If(y,x)),If(y,z)),If(x,z))')
mb = MaceCommand(goal, assumptions=ortho_boolean_lattice + [if_and,if_top,simp_caut_trans,caut_mon])
mb.build_model()
print(mb.model(format='cooked'))

Consider, for $\alpha,\beta$ non-epistemic, $(\alpha\to \Diamond\varphi)\wedge (\alpha \to (\varphi\to\beta)) \equiv (\alpha\to \Diamond\varphi) \wedge ((\alpha\wedge\varphi)\to\beta)$ (Qualified Import-Export), which is proved in the paper.

In [ ]:
goal = read_expr('(B(x) & B(z)) -> And(If(x,Diamond(y)),If(x,If(y,z))) = And(If(x,Diamond(y)),If(And(x,y),z))')
prover = Prover9Command(goal, assumptions=cond_epistemic_ortho_boolean_lattice)
prover.prove()
print(prover.proof())

In [ ]:
goal = read_expr('(B(x) & B(z)) -> And(If(x,Diamond(y)),If(x,If(y,z))) = And(If(x,Diamond(y)),If(And(x,y),z))')
mb = MaceCommand(goal, assumptions= cond_epistemic_ortho_boolean_lattice)
mb.build_model()
print(mb.model(format='cooked'))